# Programming Challenge

### 1. Introduction
The goal of this challenge is to give you a chance to show how you would solve a small programming task, while at the same time give you a glimpse into the kind of problems we come across on a regular basis.
#### 1.1. Problem

You have a number of files of different size. You also have a number of compute nodes, with available space on their hard drives; the available space may also vary widely.

Your goal is to distribute the files as evenly as possible to the nodes.  One reason for doing this might be that you have a lot of clients reading these files, and you want to spread the load of these reads as evenly as possible across the nodes.

Note that it is the space used by the files, not the free space on the disk, that you are trying to balance. The free space is just a constraint on how much data you can fit onto the disk.
#### 1.2. Hints

1. Start with biggest files first, smallest ones last.
2. Try to put files on the node that has the least amount of data added so far.
3. Think about the data structures that will make it easier to access files and nodes in the order you want.
4. A perfect solution is not necessary. Try to get something reasonably good that can be achieved in a reasonable time, even for tens of thousands of files and hundreds of nodes.
5. Although a good solution is more important than beautiful code, please try to add reasonable comments and docstrings just as you would for a real Python program.  Consider your solution as something other people will use as part of a larger system.


### 2. Details
This section gives details on what the inputs will look like, what the outputs should look like, and how the program will be invoked from the command line.
#### 2.1. Inputs

There are two input files: `files` and `nodes`.  Both have the same
structure: simple text files, with one item per line, using the
newline character `\n` to separate lines. For both files, any blank line or line beginning with the comment character ('#') should be ignored. Any other line will have two fields separated by one or more spaces. The first field will be a string and the second field will be an integer:

 For `files`, the first field is the name of the file and the second field is the size of the file (in bytes).
 For `nodes`, the first field is the name of the node and the second field is the amount of free space (in bytes).

Example of "files":

	# filename size  
	tom.dat 1024
	jerry.dat 16553
	tweety.out 12345
	elmerfudd.txt 987654321
    
Example of "nodes":

	# node-name available-space
	node1 65536
	node2 32768
#### 2.2. Output
The program should print its output to the terminal (i.e. standard output, the default for
the "print" statement). For each file, the output should have one line, with two fields separated by a space: the name of the file, and the  name of the node where it was placed by the program.
For files that didn't get placed on any node, use the word "NULL" for the node name.

Example of the output:

	tom.dat node1
	tweety.out node1
	jerry.dat node2
	


In [267]:
### install and import packages
import pandas as pd
from heapq import heappush, heappop
import heapq, argparse

In [266]:
# commandline parser
parser = argparse.ArgumentParser()
parser.add_argument("--files", help='the path of the list of file name and size file')
parser.add_argument("--nodes", help='the path of the list of node name and available space file')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--files FILES] [--nodes NODES]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/michaelwu/Library/Jupyter/runtime/kernel-33a80656-32b6-4965-9423-03d5c670911f.json


SystemExit: 2

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [258]:
### read in data
file_name = args.files
node_name = args.nodes
files_df = pd.read_csv(file_name,comment="#", delim_whitespace=True, header=None)
nodes_df = pd.read_csv(node_name,comment="#", delim_whitespace=True, header=None)

In [259]:
### reset colum names
files_df.columns=['filename','size']
nodes_df.columns=['nodes','available_space']

In [290]:
### define customized data objects
class File:
    def __init__(self, filename, size, storage_node=None):
        self.filename = filename
        self.size = size
        self.storage_node = None
    def __str__(self):
        return "{} {}".format(self.filename, "Null" if self.storage_node == None else self.storage_node.nodeName)
    def __repr__(self):
        return "filename: {} | size: {} | storage_node: {}".format(self.filename, self.size, "Null" if self.storage_node == None else self.storage_node.nodeName)
    #for this.size < other.size
    def __lt__(self, other):
        return self.size < other.size
    # this.size <= other.size
    def __le__(self, other):
        return self.size <= other.size
    # greater than
    def __gt__(self, other):
        return self.size > other.size
    def __ge__(self, other):
        return self.size >= other.size
    # add the node to storage node and return true, return false if cannot add
    def addNode(self, node):
        if node.available_space >= self.size:
            node.available_space = node.available_space - self.size
            node.used_space = self.size + node.used_space
            self.storage_node = node
            return True
        else:
            return False
    
class Node:
    def __init__(self, nodeName, available_space, used_space=0):
        self.nodeName = nodeName
        self.available_space = available_space
        self.used_space = used_space
    def __str__(self):
        return self.__repr__()
    def __repr__(self):
        return "nodeName: {} | available_space: {} | used_space: {}".format(self.nodeName, self.available_space, self.used_space)
    #for this.size < other.size
    def __lt__(self, other):
        return self.used_space < other.used_space if self.used_space != other.used_space else self.available_space < other.available_space
    # this.size <= other.size
    def __le__(self, other):
        return self.used_space <= other.used_space if self.used_space != other.used_space else self.available_space <= other.available_space
    # greater than
    def __gt__(self, other):
        return self.used_space > other.used_space if self.used_space != other.used_space else self.available_space > other.available_space
    def __ge__(self, other):
        return self.used_space >= other.used_space if self.used_space != other.used_space else self.available_space >= other.available_space

In [291]:
# put dataframe into array for sorting
files = []
nodes = []
for index, row in files_df.iterrows():
    f = File(row['filename'], row['size'])
    files.append(f)
for index, row in nodes_df.iterrows():
    n = Node(row['nodes'], row['available_space'])
    nodes.append(n)

In [292]:
### inplace heapification
heapq._heapify_max(files)
heapq._heapify_max(nodes)

In [293]:
# initialize a result list for storing results
result = []
# as long as there are still files in the heap
# pop off the first element in the file heap, call it current_file
# pop off the first element in the nodes heap, call it current_node
# if current_file add current_node is successful
#     check if the node still have space
#         if yes, push the node back to the nodes heap
# else(current_file cannot be stored in current_node)
#     as long as there are still node in the nodes heap
#         push the current node to a temporary list used to push the the current node back onto the heap
#         pop off the first node, set current_node to that node
#         if current_file can add current_node, break, otherwise, continue
#     if there are no nodes that can store this file, this file just simply would not be modified, and the field of storage node will be None
#     re-heapify nodes
# append the current_file to the list of result
while files:
    current_file = heapq.heappop(files)
    temp_nodes = []
    current_node = heapq.heappop(nodes)
    if not current_file.addNode(current_node):
        while(nodes):
            temp_nodes.append(current_node)
            current_node = heapq.heappop(nodes)
            if current_file.addNode(current_node):
                break
        nodes = nodes + temp_nodes
        heapq._heapify_max(nodes) 
        temp_nodes = []

    if current_node.available_space > 0:
        heapq.heappush(nodes, current_node)
    result.append(current_file)       

In [294]:
for file in result:
    print(file)

elmerfudd.txt Null
tom.dat node2
tweety.out node1
jerry.dat node2


In [ ]:
class File:
    def __init__(self, filename, size, storage_node=None):
        self.filename = filename
        self.size = size
        self.storage_node = None
    def __str__(self):
        return "{} {}".format(self.filename, "Null" if self.storage_node == None else self.storage_node.nodeName)
    def __repr__(self):
        return "filename: {} | size: {} | storage_node: {}".format(self.filename, self.size, "Null" if self.storage_node == None else self.storage_node.nodeName)
    #for this.size < other.size
    def __lt__(self, other):
        return self.size < other.size
    # this.size <= other.size
    def __le__(self, other):
        return self.size <= other.size
    # greater than
    def __gt__(self, other):
        return self.size > other.size
    def __ge__(self, other):
        return self.size >= other.size
    # add the node to storage node and return true, return false if cannot add
    def addNode(self, node):
        if node.available_space >= self.size:
            node.available_space = node.available_space - self.size
            node.used_space = self.size + node.used_space
            self.storage_node = node
            return True
        else:
            return False
    
class Node:
    def __init__(self, nodeName, available_space, used_space=0):
        self.nodeName = nodeName
        self.available_space = available_space
        self.used_space = used_space
    def __str__(self):
        return self.__repr__()
    def __repr__(self):
        return "nodeName: {} | available_space: {} | used_space: {}".format(self.nodeName, self.available_space, self.used_space)
    #for this.size < other.size
    def __lt__(self, other):
        return self.used_space < other.used_space if self.used_space != other.used_space else self.available_space < other.available_space
    # this.size <= other.size
    def __le__(self, other):
        return self.used_space <= other.used_space if self.used_space != other.used_space else self.available_space <= other.available_space
    # greater than
    def __gt__(self, other):
        return self.used_space > other.used_space if self.used_space != other.used_space else self.available_space > other.available_space
    def __ge__(self, other):
        return self.used_space >= other.used_space if self.used_space != other.used_space else self.available_space >= other.available_space

In [7]:
### install and import packages
from File import File
from Node import Node
import pandas as pd
from heapq import heappush, heappop
import heapq, argparse
# commandline parser
parser = argparse.ArgumentParser()
parser.add_argument("--files", help='the path of the list of file name and size file')
parser.add_argument("--nodes", help='the path of the list of node name and available space file')
args = parser.parse_args()

### read in data
file_name = args.files
node_name = args.nodes

file_df = None
nodes_df = None
try:
    files_df = pd.read_csv(file_name,comment="#", delim_whitespace=True, header=None)
    nodes_df = pd.read_csv(node_name,comment="#", delim_whitespace=True, header=None)
except e:
    print("Argument not right")

### reset colum names
files_df.columns=['filename','size']
nodes_df.columns=['nodes','available_space']

# put dataframe into array for sorting
files = []
nodes = []
for index, row in files_df.iterrows():
    f = File(row['filename'], row['size'])
    files.append(f)
for index, row in nodes_df.iterrows():
    n = Node(row['nodes'], row['available_space'])
    nodes.append(n)
    
### inplace heapification
heapq._heapify_max(files)
heapq._heapify_max(nodes)
result = []


# as long as there are still files in the heap
# pop off the first element in the file heap, call it current_file
# pop off the first element in the nodes heap, call it current_node
# if current_file add current_node is successful
#     check if the node still have space
#         if yes, push the node back to the nodes heap
# else(current_file cannot be stored in current_node)
#     as long as there are still node in the nodes heap
#         push the current node to a temporary list used to push the the current node back onto the heap
#         pop off the first node, set current_node to that node
#         if current_file can add current_node, break, otherwise, continue
#     if there are no nodes that can store this file, this file just simply would not be modified, and the field of storage node will be None
#     re-heapify nodes
# append the current_file to the list of result
def arrange(result, files, nodes):
    while files:
        current_file = heapq.heappop(files)
        temp_nodes = []
        current_node = heapq.heappop(nodes)
        if not current_file.addNode(current_node):
            while(nodes):
                temp_nodes.append(current_node)
                current_node = heapq.heappop(nodes)
                if current_file.addNode(current_node):
                    break
            nodes = nodes + temp_nodes
            heapq._heapify_max(nodes) 
            temp_nodes = []

        if current_node.available_space > 0:
            heapq.heappush(nodes, current_node)
        result.append(current_file) 
    return result
 
arrange(result, files, nodes)
# print out result as required
for file in result:
    print(file)

usage: ipykernel_launcher.py [-h] [--files FILES] [--nodes NODES]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/michaelwu/Library/Jupyter/runtime/kernel-0c61fb05-9f0c-4b08-b9eb-31e0961c8cb9.json


SystemExit: 2

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
